In [100]:
import pandas as pd
import geopandas as gpd
import fiona
import math
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster


In [101]:
fiona.drvsupport.supported_drivers['libkml'] = 'rw' # enable KML support which is disabled by default
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw' # enable KML support which is disabled by default


gdf_full = gpd.read_file("file://localhost/users/nicki/desktop/udvikling/python/data/Jordbrugsanalyser-Ansogere13.kml")

gdf_full = gdf_full[["Name", "geometry"]]

gdf = gdf_full.loc[0:1499].copy()

gdf["x"]=gdf.geometry.x
gdf["y"]=gdf.geometry.y


In [102]:
kommuner = gpd.read_file("file://localhost/users/nicki/desktop/udvikling/python/data/KOMMUNE.zip")

kommuner = kommuner[["FEAT_ID", "geometry"]]

In [103]:
kommuner=kommuner.to_crs(epsg=4326)
gdf.crs==kommuner.crs

True

Simple Interactive map
-

In [104]:
# Create a map
# Go to OpenStreetmaps, find view, and click share - 
# https://www.openstreetmap.org/#map=10/55.6964/12.2443

#Styles cartodbdark_matter  cartodbpositron   cartodbpositronnolabels   cartodbpositrononlylabels    openstreetmap    stamenterrain   stamentoner stamenwatercolor

m_1 = folium.Map(location=[55.6964,12.2443], tiles='openstreetmap', zoom_start=10)

m_1

In [105]:
# Create a map
m_2 = folium.Map(location=[56.072,8.624], tiles='cartodbpositron', zoom_start=8)


# Add points to the map
for idx, row in gdf.iterrows():
    Marker([row['y'], row['x']]).add_to(m_2)
m_2

Plotting points
-

In [106]:
#Clustering all points
# Create the map
m_3 = folium.Map(location=[56.072,8.624], tiles='cartodbpositron', zoom_start=8)

# Add points to the map
mc = MarkerCluster()
for idx, row in gdf.iterrows():
    if not math.isnan(row['x']) and not math.isnan(row['y']):
        mc.add_child(Marker([row['y'], row['x']]))

m_3.add_child(mc)


In [107]:
import numpy as np
#Bubble plot


# Create a base map
m_4 = folium.Map(location=[56.072,8.624], tiles='cartodbpositron', zoom_start=8)

def color_producer(val):
    if np.random.random_sample() <= 0.9:
        return 'forestgreen'
    else:
        return 'darkred'

def size_producer(val):
    return np.random.random_sample() * 2000

# Add a bubble map to the base map
for i in range(0,len(gdf)):
    Circle(
        location=[gdf.iloc[i]['y'], gdf.iloc[i]['x']],
        radius=size_producer(gdf.iloc[i]['Name']),
        color=color_producer(gdf.iloc[i]['Name'])).add_to(m_4)
m_4

In [108]:
#HEATMAP
# Create a base map
m_5 = folium.Map(location=[56.072,8.624], tiles='cartodbpositron', zoom_start=8)

# Add a heatmap to the base map
HeatMap(data=gdf[['y', 'x']], radius=15).add_to(m_5)

# Display the map
m_5



Choropleth maps
-

In [109]:
# GEOJason with kommuner
poly = kommuner[["FEAT_ID", "geometry"]].set_index("FEAT_ID").__geo_interface__


In [110]:
#Create a pandas dataframe with points within each poly
joined = gpd.sjoin(gdf_full, kommuner, how="inner", op='intersects')

kommuner["ansognigner_13"]=0

for i in range(0,len(kommuner)):
    kommuner["ansognigner_13"][i]=(joined.FEAT_ID==kommuner["FEAT_ID"][i]).sum()

#poly and plot_dict must share same id
plot_dict = kommuner[["FEAT_ID", "ansognigner_13"]]
plot_dict["FEAT_ID"]=plot_dict["FEAT_ID"].astype("str")
plot_dict=plot_dict.set_index("FEAT_ID")

In [116]:
# Create a base map
m_6 = folium.Map(location=[56.072,8.624], tiles='cartodbpositron', zoom_start=6)

folium.Choropleth(geo_data=poly,
           data=plot_dict["ansognigner_13"],
           key_on="feature.id",
           fill_color='YlGnBu',
           legend_name='Ansogninger',
           fill_opacity=0.4,
           line_opacity=0.7
          ).add_to(m_6)
# Display the map
m_6

Save to HTML
-

In [117]:
# Save to html
m_6.save('Ansøgninger_fordelt_på_kommuner.html')